---
# Lab Number : 1

## Title : *Data Analysis with Spark* 

## Goal : Getting Familiar with Spark workflow  

## Help:

1. Spark Programming Guide : https://spark.apache.org/docs/2.2.0/rdd-programming-guide.html
2. Spark API reference : https://spark.apache.org/docs/latest/api/python/index.html

## Datasets reference:

https://archive.ics.uci.edu/ml/datasets/bank+marketing

## Input Datasetss to be used :


## Datasets local path:

```
/spark-course/data/bank/
```

## Reading (do not yet):

[Moro et al., 2014](https://www.researchgate.net/publication/260805594_A_Data-Driven_Approach_to_Predict_the_Success_of_Bank_Telemarketing) A Data-Driven Approach to Predict the Success of Bank Telemarketing.

---

## Lab Specific Tasks

### Initialization

 * Create a SparkSession
 * Load the dataset : Bank Products Marketing

### Basic Analysis

 * Inspect the dataset and analyze it's structure (schema)
 * Report the number of columns and their names
 * Report the number of records in dataset
 * Transform implicitly numeric columns into actual numeric columns
 * Perform simple (descriptive) statistics ( max, min , average, median , stddev) on numeric columns were applicable
 
### Advanced Analysis

 * Perform simple statistics PER 'job_type' and 'marital_status'
 * Quantify 
 * Investigate and quantify correlations (if any) between features 

### Bonus

 * Propose an algorithm for predicting user's predisposition to buy a new bank marketing product
 * May too ambitious for this Lab?

### Save and Report Your Results

Once your finished save your notebook: 

1. Go File -> Save and Checkpoint
2. Email the professor (aabreua@faculty.ie.edu) your saved Notebook ( the ..ipynb file )
3. **Note : every** bulleted task above shall have it's corresponding pyspark code in your notebook. 

In [4]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

spark = SparkSession \
        .builder \
        .appName("Lab1") \
        .getOrCreate()

sc = spark.sparkContext

In [6]:
datasets_path='/spark-course/data/bank/'
bank_data=datasets_path+'bank.csv'
# Use it to load some data
df= spark \
    .read \
    .option("header","true") \
    .csv(bank_data)

In [7]:
# What is df ?
df

DataFrame["age";"job";"marital";"education";"default";"balance";"housing";"loan";"contact";"day";"month";"duration";"campaign";"pdays";"previous";"poutcome";"y"": string]

In [8]:
# ok , but this is not very ... telling , we want to see some of the data also
df.take(5)

[Row("age";"job";"marital";"education";"default";"balance";"housing";"loan";"contact";"day";"month";"duration";"campaign";"pdays";"previous";"poutcome";"y""='30;"unemployed";"married";"primary";"no";1787;"no";"no";"cellular";19;"oct";79;1;-1;0;"unknown";"no"'),
 Row("age";"job";"marital";"education";"default";"balance";"housing";"loan";"contact";"day";"month";"duration";"campaign";"pdays";"previous";"poutcome";"y""='33;"services";"married";"secondary";"no";4789;"yes";"yes";"cellular";11;"may";220;1;339;4;"failure";"no"'),
 Row("age";"job";"marital";"education";"default";"balance";"housing";"loan";"contact";"day";"month";"duration";"campaign";"pdays";"previous";"poutcome";"y""='35;"management";"single";"tertiary";"no";1350;"yes";"no";"cellular";16;"apr";185;1;330;1;"failure";"no"'),
 Row("age";"job";"marital";"education";"default";"balance";"housing";"loan";"contact";"day";"month";"duration";"campaign";"pdays";"previous";"poutcome";"y""='30;"management";"married";"tertiary";"no";1476;"y

In [9]:
# You can se how a Spark DataFrame is actually a Dataset[Row] abstraction
# Let's analyze some data
# First let's check the schema
df.printSchema()

root
 |-- "age";"job";"marital";"education";"default";"balance";"housing";"loan";"contact";"day";"month";"duration";"campaign";"pdays";"previous";"poutcome";"y"": string (nullable = true)



In [10]:
# but there seems to be something odd here there is only the 'root' node and then a flat leaf 
# with everything recorded as strings , even stuff that is certainly numeric
# so .. let's provide ourselves the schema

#### Manually Specify data schema

In [11]:
# we can specify the schema ourselves
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql import Row
fields = [ \
          StructField("age", DoubleType(), True), \
          StructField("job", StringType(), True), \
          StructField("marital", StringType(), True), \
          StructField("education", StringType(), True), \
          StructField("default", StringType(), True), \
          StructField("balance", DoubleType(), True), \
          StructField("housing", StringType(), True), \
          StructField("loan", StringType(), True), \
          StructField("contact", StringType(), True), \
          StructField("day", StringType(), True), \
          StructField("month", StringType(), True), \
          StructField("duration", IntegerType(), True), \
          StructField("campaign", IntegerType(), True), \
          StructField("pdays", IntegerType(), True), \
          StructField("previous", IntegerType(), True), \
          StructField("poutcome", StringType(), True)]

custom_schema=StructType(fields)

In [12]:
mdf= spark \
    .read \
    .option("header","true") \
    .schema(custom_schema) \
    .csv(bank_data)

In [13]:
mdf.printSchema()

root
 |-- age: double (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- balance: double (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- day: string (nullable = true)
 |-- month: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- campaign: integer (nullable = true)
 |-- pdays: integer (nullable = true)
 |-- previous: integer (nullable = true)
 |-- poutcome: string (nullable = true)



In [14]:
# This looks better
# What about inferring the Schema?